### 1. Start SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test SparkSession") \
     .getOrCreate()

In [6]:
spark.version

'2.4.4'

### 2. Load Data and Convert

In [11]:
df_in=spark.read.load("s3://lusun-bucket1/subset-compiled.csv",format="csv",header="true")

In [12]:
df_in.printSchema()

root
 |-- analysis_sample_rate: string (nullable = true)
 |-- artist_7digitalid: string (nullable = true)
 |-- artist_familiarity: string (nullable = true)
 |-- artist_hotttnesss: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_mbid: string (nullable = true)
 |-- artist_mbtags: string (nullable = true)
 |-- artist_mbtags_count: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_playmeid: string (nullable = true)
 |-- artist_terms: string (nullable = true)
 |-- artist_terms_freq: string (nullable = true)
 |-- artist_terms_weight: string (nullable = true)
 |-- audio_md5: string (nullable = true)
 |-- bars_confidence: string (nullable = true)
 |-- bars_start: string (nullable = true)
 |-- beats_confidence: string (nullable = true)
 |-- beats_start: string (nullable = true)
 |-- danceability:

In [ ]:
from pyspark.sql.types import FloatType
#convert field type
float_list=['duration','tempo','loudness']
...
# Need detail on description of fields
# Several fields are all 0, eg: 'energy' and 'danceability'

### 3. Simple Task
Year Recognition: using features of songs to predict the corresponding year.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler,StandardScaler

cate_cols=[...]
num_cols=[...]

# String Indexing for categorical features
indexers = [StringIndexer(inputCol=col, 
                          outputCol="{}_idx".format(col),handleInvalid='skip') \
                          for col in cate_cols]

# One-hot encoding for categorical features
encoders = [OneHotEncoder(inputCol="{}_idx".format(col), 
                          outputCol="{}_oh".format(col)) \
                          for col in cate_cols]
# Concat Feature Columns
assembler = VectorAssembler(inputCols = num_cols + \
                        ["{}_oh".format(col) for col in cate_cols], 
                        outputCol = "_features")
# Standardize Features
scaler = StandardScaler(inputCol='_features', 
                        outputCol='features', 
                        withStd=True, withMean=True)